In [0]:
!pip install mpi4py

#Question 1 [point to point communication] - 10 points
Create an MPI program that works for 2 processes. 
The program should:


1.   generate a random number at both processes and print it with the rank so we know which process generated which number
2.   send process 0’s random number to process 1
3. At process 1, find the larger of the two random numbers and send the larger value back to process 0
4. Receive the larger value at process 0 and print it out




In [0]:
%%writefile mpi1.py
import numpy
from mpi4py import MPI
comm = MPI.COMM_WORLD
rank = comm.Get_rank()

randNum = numpy.zeros(1) #initialized an array of length 1 looks like[0.] this needs to be done because we can only pass a buffer and numpty array is a buffer
max_value = numpy.zeros(1) #somewhere to store the values makes code cleaner

if rank == 0:
        randNum = numpy.random.random_sample(1) #draw a random number and assign it to array randnum
        print("Process/Rank", rank, "drew the number", randNum[0])
        comm.Send(randNum, dest=1) #send the random number to rank 1
        comm.Recv(max_value, source=1)
        print("The larger value between the two processes is", max_value[0])


if rank == 1:
          comm.Recv(randNum, source=0)
          max_value = numpy.asarray(randNum)
          randNum = numpy.random.random_sample(1) #draw the second 
          print("Process/Rank", rank, "drew the number", randNum[0])
          max_value = max(numpy.append(max_value, randNum, axis=None))
          comm.Send(max_value, dest=0)
 

Writing mpi1.py


In [0]:
!mpiexec --allow-run-as-root -n 2 python mpi1.py

Process/Rank 0 drew the number 0.7816656938921069
Process/Rank 1 drew the number 0.6538691065113417
The larger value between the two processes is 0.7816656938921069


#Question 2 [non-blocking communication, overlapping] - 10 points

Restructure (copy and paste to a new block) your MPI program from question 1 to use Isend and Irecv. This time have process 1 perform an Irecv before it generates its own random number. Then have it wait for the message from process 0 and continue from step 3 as in question

In [0]:
%%writefile mpi2.py
import numpy
from mpi4py import MPI
comm = MPI.COMM_WORLD
rank = comm.Get_rank()

randNum = numpy.zeros(1) #initialized an array of length 1 looks like[0.] this needs to be done because we can only pass a buffer and numpty array is a buffer
max_value = numpy.zeros(1) #somewhere to store the values makes code cleaner

if rank == 0:
        randNum = numpy.random.random_sample(1) #draw a random number and assign it to array randnum
        print("Process/Rank", rank, "drew the number", randNum[0])
        req = comm.Isend(randNum, dest=1) #send the random number to rank 1
        req = comm.Irecv(max_value, source=1)
        req.Wait()
        print("The larger value between the two processes is", max_value[0])


if rank == 1:
          req = comm.Irecv(randNum, source=0)
          req.Wait()
          max_value = numpy.asarray(randNum)
          randNum = numpy.random.random_sample(1) #draw the second 
          print("Process/Rank", rank, "drew the number", randNum[0])
          max_value = max(numpy.append(max_value, randNum, axis=None))
          comm.Isend(max_value, dest=0)

Writing mpi2.py


In [0]:
!mpiexec --allow-run-as-root -n 2 python mpi2.py

Process/Rank 0 drew the number 0.5360120499233967
Process/Rank 1 drew the number 0.07854367309369437
The larger value between the two processes is 0.5360120499233967


#Question 3 [collective communication] - 10 points
Restructure (copy and paste to a new block) your MPI program from question 1 to use collective communication instead of point to point message passing. This time our program needs to work for N processes. The program should:
generate a random number at each process and print it with the rank so we know which process generated which number
Use a reduce with the MPI.MAX operator to collect the maximum value at process 0
At process 0 print out the max value
Make sure to test this program with at least 8 processes.



In [0]:
%%writefile mpi_collect.py
import numpy
from mpi4py import MPI
comm = MPI.COMM_WORLD
rank = comm.Get_rank()

randNum = numpy.zeros(1) 
max_value = numpy.zeros(1)


randNum = numpy.random.random_sample(1) #draw a random number and assign it to array randnum this is done for every rank
print("Process/Rank", rank, "drew the number", randNum[0])
comm.Reduce(randNum, max_value, op=MPI.MAX, root=0)

if rank == 0:
    print("The larger value between the all processes is", max_value[0])


Writing mpi_collect.py


In [37]:
!mpiexec --allow-run-as-root -n 8 python mpi_collect.py

Process/Rank 1 drew the number 0.9395131673520658
Process/Rank 5 drew the number 0.03436281630597171
Process/Rank 4 drew the number 0.9634550280745199
Process/Rank 2 drew the number 0.25578268761666745
Process/Rank 0 drew the number 0.9924593187387739
Process/Rank 3 drew the number 0.19927264288548086
The larger value between the all processes is 0.9924593187387739


#Question 4 [wildcard receive] - 10 points
To allow our processes to use MPI.ANY_SOURCE effectively, we need to capture the status of the received message. This status allows us to know which process was the source of the message we just received. Here is a snippet of code that creates a Status object, passes the object into the receive function, and examines the source of the message received:

message = numpy.zeros(1)
status = MPI.Status()
comm.Recv(message, source=MPI.ANY_SOURCE, status=status)
print(status.source) # will print the process ID we received message from


Restructure (copy and paste to a new block) your MPI program from question 3 to use wildcard point to point message passing. The program should:



1.   generate a random number at each process and print it with the rank so we know which process generated which number
2.   All processes except 0 should send their random number as a message to process 0
3. Process 0 should loop P-1 times where P is the size of the mpi comm world (comm.Get_size())

    *   In each iteration use a comm.Recv with the any source wildcard
    *   Check the received value and see if it is larger than the current max
     random value, if so update the max random value and update another
      variable to track the source of the max random value

4. After all messages have been received process 0 should print the max value and the source of the max value



In [0]:
%%writefile mpi_collect2.py
import numpy
from mpi4py import MPI
comm = MPI.COMM_WORLD
rank = comm.Get_rank()

randNum = numpy.zeros(1) 
max_value = numpy.zeros(1)
status = MPI.Status()
size = comm.Get_size()
currentmax = 0

randNum = numpy.random.random_sample(1) #draw a random number and assign it to array randnum this is done for every rank
print("Process/Rank", rank, "drew the number", randNum[0]) #1
comm.Send(randNum, dest=0) 

# comm.Reduce(randNum, max_value, op=MPI.MAX, root=0)
if rank == 0:
  reduced_size = size-1
  for p in range(reduced_size):#this means 0 - 2 or 0 - p-1
    comm.Recv(max_value, source=MPI.ANY_SOURCE, status=status)
    if max_value[0] > currentmax:
      currentmax = (max_value[0]) #keep track of max value
      status_location = status.source

  print("Process {} max random number {} found at rank {}".format(rank, currentmax, status_location) )


Writing mpi_collect2.py


In [0]:
!mpiexec --allow-run-as-root -n 5 python mpi_collect2.py

Process/Rank 1 drew the number 0.27281337893079827
Process/Rank 2 drew the number 0.9449264830855757
Process/Rank 4 drew the number 0.7911176254822279
Process/Rank 0 drew the number 0.3824080589362574
Process/Rank 3 drew the number 0.642643234999984
Process 0 max random number 0.9449264830855757 found at rank 2


#Question 5 [numpy array syntax] - 10 points
Python MPI uses numpy to send and receive data as arrays. If you have not used numpy before with Python, please read through the tutorial here: http://scipy-lectures.org/intro/numpy/index.html

We won’t be using a lot of specialized code, but a numpy array can be sliced and written to. This will help with our sending and receiving of messages so that we don’t have to create a lot of copies of data. Run the following program and understand how it is writing the message (array) received at process 0 to a different slice (10:20 when it receives a message from process 1) of the maxRandNum array:


In [0]:
%%writefile lab7q5.py
import numpy
from mpi4py import MPI
comm = MPI.COMM_WORLD
rank = comm.Get_rank()
size = comm.Get_size()


maxRandNum = numpy.zeros(10*size)
randNum = numpy.random.random_sample(10)
print("Process", rank, "drew the numbers", randNum)

if rank == 0:
  status = MPI.Status()
  maxRandNum[0:10] = randNum[0:10]
  comm.Probe(source=MPI.ANY_SOURCE, status=status)
  reduced_size = size - 1
  for p in range(reduced_size):#this means 0 - 2 or 0 - p-1
    comm.Recv(maxRandNum[10*status.source:10*(status.source+1)],source=MPI.ANY_SOURCE, status=status)
  print("Process", rank, "max random number", maxRandNum.max())
else:
  comm.Send(randNum, dest=0)


Writing lab7q5.py


In [0]:
!mpiexec --allow-run-as-root -n 25 python lab7q5.py


Process 14 drew the numbers [0.35045415 0.951836   0.06377967 0.77829062 0.62160524 0.52302939
 0.09827524 0.58481994 0.01090003 0.32365137]
Process 6 drew the numbers [0.50935449 0.60698769 0.11838204 0.42267422 0.88048164 0.87466861
 0.09522862 0.20564277 0.73206301 0.26755867]Process 1 drew the numbers [0.14035583 0.75295062 0.13509872 0.07415971 0.42625042 0.65525857
 0.05379081 0.4438256  0.3781156  0.73321721]Process 5 drew the numbers [0.87622585 0.85361832 0.93527378 0.259713   0.43245503 0.20684027
 0.1101201  0.05051559 0.56103905 0.92286321]
Process 17 drew the numbers [0.69511848 0.77020271 0.34701664 0.12721676 0.33465741 0.1505734
 0.74796279 0.26992785 0.3986206  0.79603523]
Process 2 drew the numbers [0.99224927 0.01168531 0.06447787 0.89976128 0.15968675 0.65931457
 0.98459248 0.96716452 0.78306066 0.83662983]
Process 16 drew the numbers [0.44318593 0.43412918 0.95867005 0.79035419 0.51517649 0.75422124
 0.56059543 0.86990406 0.58267916 0.87551257]Process 12 drew the n

#Question 6a parallel program of given code - done in lecture

In [0]:
%%writefile conway_seq.py

import numpy, sys
from mpi4py import MPI
comm = MPI.COMM_WORLD
rank = comm.Get_rank()
P = comm.Get_size()

ROWS = 8
COLS = 8

def GETPOS(r, c):
  return r*COLS+c

def GETCOL(p):
  return p%COLS

def GETROW(p):
  return p//COLS

def D_LEFT(p):
  return p+COLS-1 if (GETCOL(p) == 0) else p-1

def D_RIGHT(p):
  return p-COLS+1 if (GETCOL(p) == COLS-1) else p+1

def D_TOP(p):
  return GETCOL(p)+((ROWS-1) * COLS) if (GETROW(p) == 0) else p-COLS

def D_BOTTOM(p):
  return GETCOL(p) if GETROW(p) == ROWS-1 else p+COLS
                     
def evolve_cell(inboard, outboard, position):
  count = inboard[D_LEFT(position)]
  count += inboard[D_RIGHT(position)]
  count += inboard[D_TOP(position)]
  count += inboard[D_BOTTOM(position)]
  count += inboard[D_LEFT(D_TOP(position))]
  count += inboard[D_RIGHT(D_TOP(position))]
  count += inboard[D_LEFT(D_BOTTOM(position))]
  count += inboard[D_RIGHT(D_BOTTOM(position))]

  outboard[position] = 1 if count == 3 or (inboard[position] and count == 2) else 0

def update_row(inboard, outboard, row):
  position = GETPOS(row, 0)
  end = position + COLS
  while position < end:
    evolve_cell(inboard, outboard, position)
    position += 1
    
    
def simulate(board, ticks, output):
  rows_per_p = ROWS // P
  status = MPI.Status()

  for i in range(ticks):
    # logic will alternate which board to read from / write to
    board_source = board[i%2]
    board_target = board[(i+1)%2]
    if output >= 2:
      print("board at tick %d:\n" % i)
      print_board(board_source)

    comm.Bcast(board_source, root=0)

    for j in range(rank*rows_per_p, (rank+1)*rows_per_p):
      update_row(board_source, board_target, j)

    if rank != 0:
      # send data to process 0
      comm.Send(board_target[rank*rows_per_p*COLS:(rank+1)*rows_per_p*COLS],dest=0)
    else:
      # receive data at process 0
      for j in range(P-1):
        # receive message
        comm.Probe(source=MPI.ANY_SOURCE, status=status)
        comm.Recv(board_target[status.source*rows_per_p*COLS:(status.source+1)*rows_per_p*COLS], source=status.source)
        # check the source of the message
        # write the received data to the correct target rows
        

def init_random(board):
  position = 0
  positions = ROWS * COLS
  while position < positions:
    board[position] = numpy.random.randint(2)
    position += 1
    
def init_glider(board):
  board[GETPOS(1,2)] = 1
  board[GETPOS(2,2)] = 1
  board[GETPOS(3,2)] = 1
  board[GETPOS(3,1)] = 1
  board[GETPOS(2,0)] = 1

    
def print_board(board):
  position = 0
  positions = ROWS * COLS
  while position < positions:
    print('x' if board[position] == 1 else ' ', end='')
    position += 1
    if position % COLS == 0:
      print()
    
    
def main():
  """
  Conway's game of life simulation. Optional arguments (must provide in order)
  GRID ticks initialization output
  GRID = number of rows and columns in the square grid
  
  example: 8 100 g 1
  would run with an 8x8 grid for 100 simulation ticks and only output the final
  
  """
  output = 1 # set to 2 to debug
  ticks = 10
  init = 'g' # glider
  if len(sys.argv) >= 2:
    global ROWS, COLS
    ROWS = int(sys.argv[1])
    COLS = ROWS
  if len(sys.argv) >= 3:
    ticks = int(sys.argv[2])
  if len(sys.argv) >= 4:
    init = sys.argv[3]
  if len(sys.argv) >= 5:
    output = int(sys.argv[4])
    
  board = [numpy.zeros(ROWS*COLS), numpy.zeros(ROWS*COLS)]
  if rank == 0:
    time_start = MPI.Wtime()
    
    if init == 'g':
      init_glider(board[0])
    else:
      init_random(board[0])
    simulate(board, ticks, output)
    
    time_end = MPI.Wtime()
    
    if output >= 1:
      print("final board:\n")
      print_board(board[0])
    print("simulation took %f seconds" % (time_end-time_start))
  else:
    simulate(board, ticks, output)
  
if __name__ == "__main__":
  main()

Overwriting conway_seq.py


In [0]:
!mpiexec --allow-run-as-root -n 1 python conway_seq.py 150 200 g 1

final board:

                                                                                                                                                      
                                                                                                                                                      
                                                                                                                                                      
                                                                                                                                                      
                                                                                                                                                      
                                                                                                                                                      
                                                                                

In [0]:
!mpiexec --allow-run-as-root -n 2 python conway_seq.py 150 200 g 1

final board:

                                                                                                                                                      
                                                                                                                                                      
                                                                                                                                                      
                                                                                                                                                      
                                                                                                                                                      
                                                                                                                                                      
                                                                                

In [34]:
!mpiexec --allow-run-as-root -n 8 python conway_seq.py 150 200 g 1

final board:

                                                                                                                                                      
                                                                                                                                                      
                                                                                                                                                      
                                                                                                                                                      
                                                                                                                                                      
                                                                                                                                                      
                                                                                

In [35]:
!mpiexec --allow-run-as-root -n 32 python conway_seq.py 150 200 g 1

final board:

                                                                                                                                                      
                                                                                                                                                      
                                                                                                                                                      
                                                                                                                                                      
                                                                                                                                                      
                                                                                                                                                      
                                                                                

In [36]:
!mpiexec --allow-run-as-root -n 1 python conway_seq.py 32 1000 r 0
!mpiexec --allow-run-as-root -n 1 python conway_seq.py 64 1000 r 0
!mpiexec --allow-run-as-root -n 1 python conway_seq.py 128 1000 r 0
!mpiexec --allow-run-as-root -n 1 python conway_seq.py 256 1000 r 0

simulation took 5.589848 seconds
simulation took 21.527863 seconds
simulation took 87.192131 seconds
simulation took 406.832320 seconds
